In [1]:
import pymysql

# Mysql DB 접속
iscream_questions = pymysql.connect(
    user='root', 
    passwd='', 
    host='192.168.1.30', 
    db='iscream_questions', 
    charset='utf8'
)

cursor = iscream_questions.cursor(pymysql.cursors.DictCursor)

sql = "SELECT question_id, body_html, list_html, explanation_html FROM questions_78786\
        UNION\
        SELECT question_id, body_html, list_html, explanation_html FROM questions_120562 WHERE etc_category1 LIKE '%중학교 1학년%'\
        UNION\
        SELECT question_id, body_html, list_html, explanation_html FROM questions_120562 WHERE etc_category1 LIKE '%중학교 2학년%'\
        UNION\
        SELECT question_id, body_html, list_html, explanation_html FROM questions_120562 WHERE etc_category1 LIKE '%중학교 3학년%'\
        UNION\
        SELECT question_id, body_html, list_html, explanation_html FROM questions_120562 WHERE etc_category1='이관' AND body_title_html NOT regexp('①|더미')"

cursor.execute(sql)
mysql_result = cursor.fetchall()

question_id_list = []
body_html_list = []
list_html_list = []
explanation_html_list = []

for i in mysql_result:
    question_id_list.append(i['question_id'])
    body_html_list.append(i['body_html'])
    list_html_list.append(i['list_html'])
    explanation_html_list.append(i['explanation_html'])
    
# print(question_id_list)
# print(body_html_list)
# print(list_html_list)
# print(explanation_html_list)

In [2]:
# html 태그 제거
def removetag(list_name):
    
    import re
    
    list_modify = []
    for i in range(len(list_name)):
        list_name[i]=re.sub('<.+?>', '', list_name[i], 0).strip()
        list_name[i] = list_name[i].replace("$$", "")
        list_name[i] = list_name[i].replace("\\hspace{1.2pt}", "")
        list_name[i] = list_name[i].replace("&nbsp;", "")
        list_name[i] = list_name[i].replace("[#NO#]", "")
        list_name[i] = list_name[i].replace("\n", "")
        list_name[i] = list_name[i].replace("\t", "")
        list_name[i] = list_name[i].replace("&lt;", " ")
        list_name[i] = list_name[i].replace("&gt;", " ")
        list_name[i] = list_name[i].replace("•", "")
        list_name[i] = list_name[i].replace("\r", "")
        list_name[i] = list_name[i].replace("　", "") # \u3000 제거
        list_name[i] = list_name[i].replace("문제시작/", "")
        list_name[i] = list_name[i].replace("해설시작/", "")
#         list_name[i] = list_name[i].replace("■▣", "")
#         list_name[i] = list_name[i].replace("", "")
        list_name[i] = list_name[i].strip()
        
        # 배점, 채점기준 내용 제거
        a = re.search("배점", list_name[i])
        if a!=None:
            list_name[i] = list_name[i].split("배점")[0]
        
        list_modify.append(list_name[i])
    
    return list_modify

body_html_list_modify = removetag(body_html_list)
list_html_list_modify = removetag(list_html_list)
explanation_html_list_modify = removetag(explanation_html_list)

# body_html_list_modify
# list_html_list_modify
# explanation_html_list_modify

In [95]:
total_list = []

for i in range(len(body_html_list_modify)):
    tot = body_html_list_modify[i] + " " + list_html_list_modify[i] + " " + explanation_html_list_modify[i]
    if '  ' != tot:
        total_list.append(tot)

print(len(total_list))

103414


In [96]:
def createtxt(list_name):
    
    import datetime
    
    basename = "createtxt"
    suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
    filename = "_".join([basename, suffix])
    
    f = open('{}.txt'.format(filename), 'w', encoding='utf-8')
    
    for i in range(len(list_name)):
        f.write('%s\n' %list_name[i])
    f.close()

createtxt(total_list)

## WordExtractor 사용

In [125]:
from soynlp.word import WordExtractor
# from soynlp.utils import DoublespaceLineCorpus
from soynlp.tokenizer import LTokenizer

# file_path = 'your file path'
# corpus = DoublespaceLineCorpus(file_path, iter_sent=True)

word_extractor = WordExtractor(min_frequency=20,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(total_list)
words = word_extractor.extract()

## 단어 점수 중 Forward cohesion 의 점수만을 이용
cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

training was done. used memory 2.211 Gbse memory 2.278 Gb
all cohesion probabilities was computed. # words = 37446
all branching entropies was computed # words = 219148
all accessor variety was computed # words = 219148


In [136]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

입니다.     (12396, 0.905, 4.674)
따라서     (23831, 0.951, 4.484)
합니다.     (8167, 0.669, 4.670)
있습니다.     (9096, 0.678, 4.637)
했습니다.     (400, 0.814, 4.447)
였습니다.     (21, 0.801, 4.455)
에서     (16731, 0.989, 4.014)
으로     (1850, 0.998, 3.931)
됩니다.     (2308, 0.971, 3.883)
구하시오.     (8732, 0.738, 4.131)
많습니다.     (899, 0.574, 4.349)
)입니다.     (166, 0.520, 4.438)
누구입니까?     (427, 0.754, 4.066)
까지     (1141, 0.971, 3.787)
한다.     (5615, 0.891, 3.823)
.따라서     (30, 0.462, 4.449)
cm     (13843, 0.965, 3.699)
필요한     (1644, 0.729, 3.964)
동안     (4399, 0.372, 4.632)
것입니다.     (2138, 0.481, 4.347)
갔습니다.     (186, 0.817, 3.812)
보세요.     (10211, 0.705, 3.959)
큽니다.     (929, 0.978, 3.628)
필요합니다.     (491, 0.692, 3.972)
)따라서     (136, 0.391, 4.506)
m입니다.     (238, 0.441, 4.343)
쓰시오.     (6077, 0.811, 3.730)
cm입니다.     (960, 0.582, 4.049)
개입니까?개     (1304, 0.496, 4.201)
개입니다.     (1644, 0.441, 4.315)


In [123]:
sent = "{}".format(total_list[29375])

print(tokenizer.tokenize(sent))

['도형', '가는', '정사각형', '이고,', '도형', '나는', '직사각형', '입니다.', '두', '도형', '중', '네', '변의', '길이', '의', '합이', '더', '긴', '것을', '찾아', '기호를', '쓰시오.', '가', ':', '13+1', '3+13+13=52(cm)나', ':', '6+9+6+9=30(cm)따라서', '네', '변의', '길이', '의', '합이', '더', '긴', '도형', '은', '가입니다.']


In [124]:
words['도형']

Scores(cohesion_forward=0.5045571557781953, cohesion_backward=0.015538033395176253, left_branching_entropy=3.2844340860587, right_branching_entropy=2.246065650381712, left_accessor_variety=109, right_accessor_variety=57, leftside_frequency=7252, rightside_frequency=201)

## LRNounExtractor_2 사용

In [138]:
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer

noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(total_list)

## Cohesion 점수 + 명사 점수 사용
# noun_scores = {noun:score.score for noun, score in nouns.items()}
# combined_scores = {noun:score + cohesion_score.get(noun, 0)
#     for noun, score in noun_scores.items()}
# combined_scores.update(
#     {subword:cohesion for subword, cohesion in cohesion_score.items()
#     if not (subword in combined_scores)}
# )

# tokenizer = LTokenizer(scores=combined_scores)

## 명사 점수 사용
noun_scores = {noun:score.score for noun, score in nouns.items()}
tokenizer = LTokenizer(scores=noun_scores)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 183343 from 103414 sents. mem=2.034 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=9305121, mem=2.106 Gb
[Noun Extractor] batch prediction was completed for 42796 words
[Noun Extractor] checked compounds. discovered 44314 compounds
[Noun Extractor] postprocessing detaching_features : 53936 -> 47411
[Noun Extractor] postprocessing ignore_features : 47411 -> 47328
[Noun Extractor] postprocessing ignore_NJ : 47328 -> 47271
[Noun Extractor] 47271 nouns (44314 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=2.186 Gb                    
[Noun Extractor] 53.08 % eojeols are covered


In [106]:
nouns['중근']

NounScore(frequency=302, score=1.0)

In [107]:
sent = "어떤 수를 3배 하여 7을 뺀 수는 어떤 수의 2배보다 5만큼 크다. 어떤 수는?"
print(tokenizer.tokenize(sent))

['어떤', '수를', '3배', '하여', '7을', '뺀', '수는', '어떤', '수의', '2배', '보다', '5만큼', '크다.', '어떤', '수는', '?']


## MaxScoreTokenizer

In [143]:
from soynlp.tokenizer import MaxScoreTokenizer

scores = {'보다 크고': 1.0, '일차방정식':0.5, '일차 방정식':0.5}
tokenizer = MaxScoreTokenizer(scores=scores)

print(tokenizer.tokenize('다음 일차 방정식의 해의 개수는'))

['다음', '일차', '방정식의', '해의', '개수는']


## 명사만으로 문제 구성

In [99]:
tokenized_list = []
for i in range(len(total_list)):
    temp = []
    temp2 = []
    temp.append(tokenizer.tokenize(total_list[i]))
    for j in range(len(temp[0])):
        if temp[0][j] in nouns:
            temp2.append(temp[0][j])
    tokenized_list.append(temp2)

print(len(tokenized_list))

103414


In [100]:
from gensim.models import Word2Vec

result = tokenized_list

model = Word2Vec(result, 
                 size=100,         # 차원
                 window=5,         # 앞뒤 고려 단어수
                 min_count=10,     # 최소 중복 개수
                 workers=4,        # CPU 코어
                 sg=1,             # CBOW / Skip-Gram
                 iter = 5,         # iteration
                 sample = 0.001)   # 빈번하게 등장하는 단어 다운샘플링

model.init_sims(replace=True)

In [104]:
model.wv.most_similar(positive="중근", topn=5)

[('이차방정식', 0.8205578327178955),
 ('완전제곱식', 0.7952829003334045),
 ('판별식', 0.7654517889022827),
 ('(중근)', 0.7142831087112427),
 ('공통인수', 0.7140236496925354)]

## 학습된 토크나이저를 이용하여 문서를 희소행렬로 변환

In [19]:
from soynlp.vectorizer import BaseVectorizer
from soynlp.utils import DoublespaceLineCorpus

vectorizer = BaseVectorizer(
    tokenizer=tokenizer,
    min_tf=0,
    max_tf=10000,
    min_df=0,
    max_df=1.0,
    stopwords=None,
    lowercase=True,
    verbose=True
)

file_path = './questions/total_list.txt'
corpus = DoublespaceLineCorpus(file_path, iter_sent=True)

corpus.iter_sent = False
x = vectorizer.fit_transform(corpus)

scanning was done                                        
121023 terms are recognized
transforming docs to term frequency marix was done


In [35]:
x

<103429x121023 sparse matrix of type '<class 'numpy.intc'>'
	with 2614378 stored elements in Compressed Sparse Row format>

In [25]:
vectorizer.encode_a_doc_to_list('다음 일차방정식의 해를 구하시오')

[353, 718, 14620]

In [26]:
vectorizer.decode_from_list([353, 718, 14620])

['일차방정식', '해를', '구하시오']

## Tagger

In [137]:
pos_dict = {
#     'Adverb': {'모두'}, 
    'Noun': {'문자', '일차방정식', '상수'},
#     'Josa': {'을'},
#     'Verb': {'고르면'},
#     'Adjective': {'예쁜', '예쁘다'},
#     'Exclamation': {'우와'}    
}

from soynlp.postagger import Dictionary
from soynlp.postagger import LRTemplateMatcher
from soynlp.postagger import LREvaluator
from soynlp.postagger import SimpleTagger
from soynlp.postagger import UnknowLRPostprocessor

dictionary = Dictionary(pos_dict)
generator = LRTemplateMatcher(dictionary)    
evaluator = LREvaluator()
postprocessor = UnknowLRPostprocessor()
tagger = SimpleTagger(generator, evaluator, postprocessor)

sent = '다음 중 문자를 사용한 식을 모두 고르면?'
print(tagger.tag(sent))

[('문자', 'Noun'), ('를', None)]
